The particle dynamics in a general velocity field is given by $ \mathbf{u}(\mathbf{x}, t) = \begin{pmatrix} u(\mathbf{x}, t) \\ v(\mathbf{x}, t) \\  w(\mathbf{x}, t) \end{pmatrix} $. 
This function evaluates the laplacian of the velocity field, $ \Delta \mathbf{u}(\mathbf{x}, t)  = \begin{pmatrix} \Delta u(\mathbf{x}, t) \\ \Delta v(\mathbf{x}, t) \\  \Delta w(\mathbf{x}, t) \end{pmatrix}$, at point $ \mathbf{x} $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| X | array (NY, NX, NZ) | X-meshgrid|
| Y | array (NY, NX, NZ) | Y-meshgrid|
| Z | array (NY, NX, NZ) | Y-meshgrid|
| U | array (NY, NX, NZ, Nt) | U velocity  |
| V | array (NY, NX, NZ, Nt) | V velocity |
| W | array (NY, NX, NZ, Nt) | W velocity |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| boundary_layer | float | Specify the value at the boundaries |
| time_data | array(Nt, ) | If the flow is unsteady,  provide time-measurements too|
| Lap_u, Lap_v, Lap_w  | arrays of (Npoints, ) | Components of the Laplacian |

In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add integration folder to current working path
sys.path.append(parent_directory+"/integration")

In [ ]:
# Import numpy
import numpy as np

# import function to compute gradient of velocity-field
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# Import interpolation function for steady flow field
from ipynb.fs.defs.Interpolant import interpolant_steady, interpolant_unsteady

In [1]:
def laplacian_velocity(t, X, Y, Z, U, V, W, periodic, bool_unsteady, boundary_layer, time_data):
    '''
    Compute the Laplacian of the velocity field (component-wise) given by gridded data. 
    
    Parameters:
        t: float, time
        X: array (NY, NX, NZ), X-meshgrid
        Y: array (NY, NX, NZ), Y-meshgrid
        Z: array (NY, NX, NZ), Y-meshgrid
        U: array (NY, NX, NZ, Nt), U velocity
        V: array (NY, NX, NZ, Nt), V velocity
        W: array (NY, NX, NZ, Nt), W velocity 
        periodic: list (3,), periodic[0]: periodicity in x
                             periodic[1]: periodicity in y
                             periodic[2]: periodicity in z
        bool_unsteady: bool, specifies if velocity field is unsteady/steady
        boundary_layer: float, specifies the value at the boundaries
        time_data: array(Nt, ), If the flow is unsteady,  provide time-measurements too

    Returns:
        Lap_u, Lap_v, Lap_w: arrays of (Npoints, ), u, v, and w components of the Laplacian
    '''
    # Set nan values to zero (in case there are any) so that we can apply interpolant. 
    # Interpolant does not work if the array contains nan values. 
    U[np.isnan(U)] = 0
    V[np.isnan(V)] = 0
    W[np.isnan(W)] = 0

    # Interpolate velocity data using cubic spatial interpolation
    if bool_unsteady:
        
        Interpolant = interpolant_unsteady(X, Y, Z, U, V, W, time_data)
        
    else:
        
        Interpolant = interpolant_steady(X, Y, Z, U, V, W)

    Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object; Interpolant for u-velocity
    Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object; Interpolant for v-velocity
    Interpolant_w = Interpolant[2] # RectangularBivariateSpline-object; Interpolant for w-velocity
    
    # grid-spacing
    dx = np.min(np.diff(X[0,:,0]))
    dy = np.min(np.diff(Y[:,0,0]))
    dz = np.min(np.diff(Z[0,0,:]))
    
    # auxiliary grid-spacing = data grid-spacing
    aux_grid = [dx, dy, dz]
    
    print("Compute 1st order spatial derivative...")
    
    # 1st order spatial derivatives on meshgrid
    x_lap = np.array([X.ravel(), Y.ravel(), Z.ravel()])
    grad_vel = gradient_velocity(t, x_lap, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid)    
    grad_vel = grad_vel.reshape(3,3,X.shape[0], X.shape[1], X.shape[2])
    
    dudx = grad_vel[0,0,:,:,:].copy()
    dvdx = grad_vel[1,0,:,:,:].copy()
    dwdx = grad_vel[2,0,:,:,:].copy()
    
    dudy = grad_vel[0,1,:,:,:].copy()
    dvdy = grad_vel[1,1,:,:,:].copy()
    dwdy = grad_vel[2,1,:,:,:].copy()
    
    dudz = grad_vel[0,2,:,:,:].copy()
    dvdz = grad_vel[1,2,:,:,:].copy()
    dwdz = grad_vel[2,2,:,:,:].copy()
    
    # Interpolate d-dx data using cubic spatial interpolation
    Interpolant = interpolant_steady(X, Y, Z, dudx, dvdx, dwdx)

    Interpolant_dudx = Interpolant[0] # RectangularBivariateSpline-object; Interpolant for dudx
    Interpolant_dvdx = Interpolant[1] # RectangularBivariateSpline-object; Interpolant for dvdx
    Interpolant_dwdx = Interpolant[2] # RectangularBivariateSpline-object; Interpolant for dwdx
    
    # Interpolate d-dy data using cubic spatial interpolation
    Interpolant = interpolant_steady(X, Y, Z, dudy, dvdy, dwdy)

    Interpolant_dudy = Interpolant[0] # RectangularBivariateSpline-object; Interpolant for dudy
    Interpolant_dvdy = Interpolant[1] # RectangularBivariateSpline-object; Interpolant for dvdy
    Interpolant_dwdy = Interpolant[2] # RectangularBivariateSpline-object; Interpolant for dwdy
    
    # Interpolate d-dz data using cubic spatial interpolation
    Interpolant = interpolant_steady(X, Y, Z, dudz, dvdz, dwdz)

    Interpolant_dudz = Interpolant[0] # RectangularBivariateSpline-object; Interpolant for dudz
    Interpolant_dvdz = Interpolant[1] # RectangularBivariateSpline-object; Interpolant for dvdz
    Interpolant_dwdz = Interpolant[2] # RectangularBivariateSpline-object; Interpolant for dwdz
    
    print("Compute 2nd order spatial derivative...")
    
    # 2nd order spatial derivatives on meshgrid
    grad_d_dx = gradient_velocity(t, x_lap, X, Y, Z, Interpolant_dudx, Interpolant_dvdx, Interpolant_dwdx, periodic, False, aux_grid)
    grad_d_dx = grad_d_dx.reshape(3,3,X.shape[0], X.shape[1], X.shape[2])
    
    grad_d_dy = gradient_velocity(t, x_lap, X, Y, Z, Interpolant_dudy, Interpolant_dvdy, Interpolant_dwdy, periodic, False, aux_grid)
    grad_d_dy = grad_d_dy.reshape(3,3,X.shape[0], X.shape[1], X.shape[2])
    
    grad_d_dz = gradient_velocity(t, x_lap, X, Y, Z, Interpolant_dudz, Interpolant_dvdz, Interpolant_dwdz, periodic, False, aux_grid)
    grad_d_dz = grad_d_dz.reshape(3,3,X.shape[0], X.shape[1], X.shape[2])
    
    dduddx = grad_d_dx[0,0,:,:,:]
    ddvddx = grad_d_dx[1,0,:,:,:]
    ddwddx = grad_d_dx[2,0,:,:,:]
    
    dduddy = grad_d_dy[0,1,:,:,:]
    ddvddy = grad_d_dy[1,1,:,:,:]
    ddwddy = grad_d_dy[2,1,:,:,:]
    
    dduddz = grad_d_dz[0,2,:,:,:]
    ddvddz = grad_d_dz[1,2,:,:,:]
    ddwddz = grad_d_dz[2,2,:,:,:]
    
    Lap_u = dduddx+dduddy+dduddz # Laplacian of u
    Lap_v = ddvddx+ddvddy+ddvddz # Laplacian of v
    Lap_w = ddwddx+ddwddy+ddwddz # Laplacian of w
    
    # Interpolate laplacian of velocity data using cubic spatial interpolation
    Interpolant = interpolant_steady(X, Y, Z, Lap_u, Lap_v, Lap_w)

    Interpolant_lap_u = Interpolant[0] # RectangularBivariateSpline-object
    Interpolant_lap_v = Interpolant[1] # RectangularBivariateSpline-object
    Interpolant_lap_w = Interpolant[2] # RectangularBivariateSpline-object
    
    from tqdm.notebook import tqdm
    
    # If boundary conditions apply
    for i in range(X.shape[0]):
        for ii in range(Y.shape[1]):
            for iii in range(Z.shape[2]):
                if periodic[0] == False:
                    
                    boundary_x_right = X[i,-1,iii] - boundary_layer
                    boundary_x_left = X[i,0,iii] + boundary_layer
                    x = X[i,ii,iii]
                    
                    if boundary_x_right < x < X[i,-1,iii]:
                        Lap_u[i,ii,iii] = Interpolant_lap_u(np.array([Y[i,ii,iii], boundary_x_right, Z[i,ii,iii]]))*((x-1)/(boundary_x_right-1))**2
                        Lap_v[i,ii,iii] = Interpolant_lap_v(np.array([Y[i,ii,iii], boundary_x_right, Z[i,ii,iii]]))*((x-1)/(boundary_x_right-1))**2
                        Lap_w[i,ii,iii] = Interpolant_lap_w(np.array([Y[i,ii,iii], boundary_x_right, Z[i,ii,iii]]))*((x-1)/(boundary_x_right-1))**2
                    
                    elif X[i,0,iii] < x < boundary_x_left:
                        Lap_u[i,ii,iii] = Interpolant_lap_u(np.array([Y[i,ii,iii], boundary_x_left, Z[i,ii,iii]]))*((x+1)/(boundary_x_left+1))**2
                        Lap_v[i,ii,iii] = Interpolant_lap_v(np.array([Y[i,ii,iii], boundary_x_left, Z[i,ii,iii]]))*((x-1)/(boundary_x_left-1))**2
                        Lap_w[i,ii,iii] = Interpolant_lap_w(np.array([Y[i,ii,iii], boundary_x_left, Z[i,ii,iii]]))*((x-1)/(boundary_x_left-1))**2
    
                if periodic[1] == False:
                    
                    boundary_y_up = Y[-1,ii,iii] - boundary_layer
                    boundary_y_down = Y[0,ii,iii] + boundary_layer
                    y = Y[i,ii,iii]
                    
                    if boundary_y_up < y < Y[-1,ii,iii]:
                        Lap_u[i,ii,iii] = Interpolant_lap_u(np.array([boundary_y_up, X[i,ii,iii], Z[i,ii,iii]]))*((y-1)/(boundary_y_up-1))**2
                        Lap_v[i,ii,iii] = Interpolant_lap_v(np.array([boundary_y_up, X[i,ii,iii], Z[i,ii,iii]]))*((y-1)/(boundary_y_up-1))**2
                        Lap_w[i,ii,iii] = Interpolant_lap_w(np.array([boundary_y_up, X[i,ii,iii], Z[i,ii,iii]]))*((y-1)/(boundary_y_up-1))**2
                    
                    elif Y[0,ii,iii] < y < boundary_y_down:
                        Lap_u[i,ii,iii] = Interpolant_lap_u(np.array([boundary_y_down, X[i,ii,iii], Z[i,ii,iii]]))*((y+1)/(boundary_y_down+1))**2
                        Lap_v[i,ii,iii] = Interpolant_lap_v(np.array([boundary_y_down, X[i,ii,iii], Z[i,ii,iii]]))*((y+1)/(boundary_y_down+1))**2
                        Lap_w[i,ii,iii] = Interpolant_lap_w(np.array([boundary_y_down, X[i,ii,iii], Z[i,ii,iii]]))*((y+1)/(boundary_y_down+1))**2
                        
                if periodic[2] == False:
                    
                    boundary_z_forw = Z[i,ii,-1] - boundary_layer
                    boundary_z_back = Z[i,ii,0] + boundary_layer
                    z = Z[i,ii,iii]
                    
                    if boundary_z_forw < z < Z[i,ii,-1]:
                        Lap_u[i,ii,iii] = Interpolant_lap_u(np.array([Y[i,ii,iii], X[i,ii,iii], z]))*((z-1)/(boundary_z_forw-1))**2
                        Lap_v[i,ii,iii] = Interpolant_lap_v(np.array([Y[i,ii,iii], X[i,ii,iii], z]))*((z-1)/(boundary_z_forw-1))**2
                        Lap_w[i,ii,iii] = Interpolant_lap_w(np.array([Y[i,ii,iii], X[i,ii,iii], z]))*((z-1)/(boundary_z_forw-1))**2
                    
                    elif Z[i,ii,0] < z < boundary_z_back:
                        Lap_u[i,ii,iii] = Interpolant_lap_u(np.array([Y[i,ii,iii], X[i,ii,iii], z]))*((z+1)/(boundary_forw_back+1))**2
                        Lap_v[i,ii,iii] = Interpolant_lap_v(np.array([Y[i,ii,iii], X[i,ii,iii], z]))*((z+1)/(boundary_forw_back+1))**2
                        Lap_w[i,ii,iii] = Interpolant_lap_w(np.array([Y[i,ii,iii], X[i,ii,iii], z]))*((z+1)/(boundary_forw_back+1))**2

    return Lap_u, Lap_v, Lap_w   